In [ ]:
print("ok")

In [ ]:
%pip install openai twilio flask

In [ ]:
%pip install python-dotenv


In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client

# Set environment variables for your credentials
# Read more at http://twil.io/secure

account_sid = "AC25ff99962f5f1c10f4610882da6d8652"
auth_token = "1a65fada9d981730798f8f8c27d35f67"
client = Client(account_sid, auth_token)

call = client.calls.create(
  url="http://demo.twilio.com/docs/voice.xml",
  to="+923464056076",
  from_="+16813666079"
)

print(call.sid)

In [ ]:
import os
from dotenv import load_dotenv
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse

# Load environment variables
load_dotenv()

# Get Twilio credentials from .env file
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")

app = Flask(__name__)

@app.route("/voice", methods=['POST'])
def voice():
    """Handle incoming voice calls"""
    response = VoiceResponse()
    response.say("Hello! Welcome to our AI Sales Agent. How can I assist you today?", voice='alice', language='en-US')
    
    # Record customer input and send it for processing
    response.record(timeout=5, transcribe=True, action="/process")
    
    return str(response)

@app.route("/process", methods=['POST'])
def process_voice():
    """Process the transcribed speech"""
    customer_speech = request.form.get("TranscriptionText", "Sorry, I couldn't understand.")
    
    response = VoiceResponse()
    response.say(f"You said: {customer_speech}. I will process your request shortly.", voice='alice')

    return str(response)

# if __name__ == "__main__":
#     app.run(port=5000, debug=True)


In [ ]:
%pip install pyngrok


https://3fb6-39-46-137-200.ngrok-free.app

In [ ]:
from twilio.rest import Client
import os
from dotenv import load_dotenv
from flask import Flask, request, jsonify

# Load environment variables
load_dotenv()

# Twilio credentials
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

app = Flask(__name__)

@app.route("/make-call", methods=['POST'])
def make_call():
    """Initiate an outbound call"""
    data = request.json
    customer_number = data.get("customer_number")

    if not customer_number:
        return jsonify({"error": "Missing customer number"}), 400

    call = client.calls.create(
        to=customer_number,
        from_=TWILIO_PHONE_NUMBER,
        url="https://abc123.ngrok.io/outbound-voice"
    )

    return jsonify({"message": "Call initiated", "call_sid": call.sid})


@app.route("/outbound-voice", methods=['POST'])
def outbound_voice():
    """Handle outbound call responses"""
    from twilio.twiml.voice_response import VoiceResponse
    response = VoiceResponse()
    response.say("Hello! This is your AI Sales Agent calling. How can I assist you today?", voice='alice', language='en-US')
    
    return str(response)

if __name__ == "__main__":
    app.run(port=5000, debug=True)


In [ ]:
%pip install -U openai-whisper

In [ ]:
%pip install elevenlabs


In [ ]:
import whisper
from elevenlabs import generate, play

# Load Whisper model
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

# Text-to-Speech function
def text_to_speech(text):
    audio = generate(text=text, voice="Adam", model="eleven_multilingual_v2")
    play(audio)

# Test transcription (Replace 'sample.wav' with an actual file path)
#audio_text = transcribe_audio("sample.wav")
#print("Transcribed Text:", audio_text)

# Test TTS
test_text = "Hello! How can I assist you today?"
text_to_speech(test_text)


In [1]:
print("feb 24")

feb 24


In [4]:
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse

app = Flask(__name__)

@app.route("/voice", methods=["POST"])
def voice():
    """Handles incoming calls and responds with the company name"""
    response = VoiceResponse()
    response.say("Hello! Welcome to XYZ Company. How can I assist you today?", voice="alice")
    return str(response)

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Feb/2025 14:48:59] "POST /voice HTTP/1.1" 200 -


In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Initialize embeddings model
embeddings = OpenAIEmbeddings()

# Sample Business Knowledge
documents = [
    Document(page_content="Our product is a cloud-based CRM that helps manage customer interactions."),
    Document(page_content="We offer a 14-day free trial with no credit card required."),
    Document(page_content="Our pricing starts at $49 per month for basic features."),
]

# Convert documents into embeddings and store in FAISS
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("faiss_index")


C:\Users\Waleed\AppData\Local\Temp\ipykernel_12992\1009501427.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [2]:


vector_store = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

def retrieve_info(query):
    results = vector_store.similarity_search(query, k=3)  # Retrieve top 3 relevant docs
    return "\n".join([doc.page_content for doc in results])


In [6]:
from flask import Flask, request, Response
from twilio.twiml.voice_response import VoiceResponse

app = Flask(__name__)

@app.route("/voice", methods=["POST"])
def voice():
    response = VoiceResponse()

    # Enable speech recognition
    response.say("Hello, welcome to [Your Company]. How can I help you today?", voice="alice")
    response.gather(input="speech", action="/process_speech", timeout=15)  # Wait for speech

    return Response(str(response), mimetype="application/xml")
@app.route("/process_speech", methods=["POST"])
def process_speech():
    print("Incoming Speech Data:", request.form)  # Log the request
    
    user_input = request.form.get("SpeechResult")  # Capture what Twilio heard
    print(f"User said: {user_input}")  # Print in console for debugging

    response = VoiceResponse()

    if user_input:
        response_text = retrieve_info(user_input)  # Process the input
    else:
        response_text = "I'm sorry, I didn't understand that."

    response.say(response_text, voice="alice")  # Twilio should speak back
    
    return Response(str(response), mimetype="application/xml")

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Feb/2025 15:27:39] "POST /voice HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2025 15:28:20] "POST /voice HTTP/1.1" 200 -
